In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Data/pulse_fates.csv")

# Extinction
df['ext'] = np.nan
ext_mask = (df['fate'] == 'failure') & (df['channel_length'] - df['track_end_position'] > 30)
df.loc[ext_mask, 'ext'] = df.loc[ext_mask, 'track_end']

# Choas
df['chaos'] = df['significant_split_time']

In [ ]:
def get_results(df):
    df_results = df.loc[:, ['ext', 'chaos']]
    df_results['Time'] = df_results[['ext', 'chaos']].min(axis=1)
    df_results['Event'] = df_results.idxmin(axis=1)

    mean_t = df_results['Time'].mean()
    events = df_results['Event'].value_counts().reindex(['ext', 'chaos']).fillna(0).astype(int)
    return events, mean_t

events, mean_t = get_results(df)

In [ ]:
def l_value(n_event, mean_t, n, T, events):
    l = n_event / (n_event * mean_t + (n - n_event) * T)
    l_chaos = events['chaos'] / n_event * l
    l_ext = events['ext'] / n_event * l
    return l, l_chaos, l_ext

n_events = events.sum()
total_time = 1200                       # Total time of simulation
n = len(df)                             # Number of simulation
l, l_chaos, l_ext = l_value(n_events, mean_t, n, total_time, events)

In [ ]:
l, l_chaos, l_ext